In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Add, Input
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, AveragePooling2D
from keras import regularizers
import os
from keras.initializers import glorot_uniform

batch_size = 32
num_classes = 10
epochs = 50
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x, y), (x_test, y_test) = cifar10.load_data()
x_train = x[0:40000]
y_train = y[0:40000]

x_val = x[40000:50000]
y_val = y[40000:50000]

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'validation samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (40000, 32, 32, 3)
40000 train samples
10000 validation samples
10000 test samples


In [0]:
def identity_block(X, f, filters, stage, block):
  #define name stage
  conv_name_base = 'conv' + str(stage)  + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'
  F1, F2, F3 = filters
  X_shortcut = X
  
  # stage a
  X = Conv2D(F1, (1,1), strides = (1, 1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
  X = Activation('relu')(X)
  
  # stage b
  
  X = Conv2D(F2, (f,f), strides = (1, 1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
  X = Activation('relu')(X)
  
  # stage c
  
  X = Conv2D(F3, (1, 1), strides = (1, 1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)
  
  X = Add()([X_shortcut, X])
  X = Activation('relu')(X)
             
  return X

In [0]:
def convolutional_block(X, f, filters, stage, block, s = 2):
  #define name stage 
  conv_name_base = 'conv' + str(stage) + block + 'branch'
  bn_name_base = 'bn' +  str(stage) + block + '_branch'
  F1, F2, F3 = filters
  
  X_shortcut = X
  
  
  
  # stage a
  
  X = Conv2D(F1, (1, 1), strides = (s, s), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
  X = Activation('relu')(X)
  
  # stage b
  X = Conv2D(F2, (f, f), strides = (1, 1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
  X = Activation('relu')(X)
  
  # stage c
  X = Conv2D(F3, (1, 1), strides = (1, 1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)
   
  
  
  X_shortcut = Conv2D(F3, (1, 1), strides = (s, s), padding = 'valid', name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
  X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)
  
  X = Add()([X_shortcut, X])
  X = Activation('relu')(X)
  
  return X

In [0]:
class StopTraining(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = {}):
    if(logs.get('val_acc') >  0.85):
      print("\nReached 85% on validation set!!! So stop training !!!\n")
      self.model.stop_training = True;
      
      
      
StopTraining = StopTraining()

In [0]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_val /= 255
x_test /= 255


print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

Using real-time data augmentation.


In [0]:

def model5(input_shape, classes):
  X_input = Input(input_shape)
  X = ZeroPadding2D((3, 3))(X_input)
  
  
  # stage 1
  
  X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
  X = Activation('relu')(X)
  X = Dropout(0.1)(X)
  X = MaxPooling2D((3, 3), strides = (2, 2))(X)
  
  # stage 2
  X = convolutional_block(X, f = 3, filters = [32, 32, 128], stage = 2, block = 'a', s = 1)
  X = Dropout(0.2)(X)
  X = identity_block(X, 3, [32, 32, 128], stage = 2, block = 'b')
  X = Dropout(0.2)(X)

  # stage 3
  X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 3, block = 'a', s = 1)
  X = Dropout(0.3)(X)
  X = identity_block(X, 3, [64, 64, 256], stage = 3, block = 'b')
  X = Dropout(0.3)(X)
  X = identity_block(X, 3, [64, 64, 256], stage = 3, block = 'c')
  X = Dropout(0.3)(X)

  # stage 4
  X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 4, block = 'a', s = 1)
  X = Dropout(0.4)(X)
  
  
  X = AveragePooling2D((2, 2))(X)
  
  X = Flatten()(X)
  X = Dense(256, activation = 'relu', name = 'fc' + str(256),kernel_regularizer=regularizers.l2(0.01), kernel_initializer = glorot_uniform(seed = 0))(X)
  X = Dropout(0.4)(X)
  
  X = Dense(classes, activation = 'softmax', name = 'fc' + str(classes),kernel_regularizer=regularizers.l2(0.01), kernel_initializer = glorot_uniform(seed = 0))(X)
  
  
  # Create Model 
  
  model = Model(input = X_input, output = X, name = 'model3')
  
  return model

In [0]:
model5 = model5(input_shape = (32, 32, 3), classes = 10)
model5.summary()

opt = keras.optimizers.Adam(lr=0.009, beta_1=0.9, beta_2=0.999, amsgrad=False)

model5.compile(loss='categorical_crossentropy',
               optimizer = opt,
               metrics = ['acc'])

print('Using real-time data augmentation.')

model5.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=64),
                        epochs=300,
                        validation_data=(x_val, y_val),
                        callbacks = [StopTraining],
                        workers=4,
                        verbose=2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="model3", inputs=Tensor("in..., outputs=Tensor("fc...)`


Model: "model3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_5 (ZeroPadding2D (None, 38, 38, 3)    0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        zero_padding2d_5[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
_____________________________________________________________________________________________

In [0]:
# Save model and weights
save_dir = '/content/drive/My Drive/Colab Notebook'
model5_name = 'keras_cifar10_trained_model_v6.h5' 
model_path = os.path.join(save_dir, model5_name)
model5.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model5.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at /content/drive/My Drive/Colab Notebook/keras_cifar10_trained_model_v6.h5 
10000/10000 [==============================] - 4s 402us/step
Test loss: 0.8704138153076172
Test accuracy: 0.8097


In [0]:
from keras.models import load_model

model5 = load_model(model_path)


opt = keras.optimizers.Adam(lr=0.0065, beta_1=0.9, beta_2=0.999, amsgrad=False)

model5.compile(loss='categorical_crossentropy',
               optimizer = opt,
               metrics = ['acc'])

print('Using real-time data augmentation.')


model5.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=64),
                        epochs=300,
                        validation_data=(x_val, y_val),
                        callbacks = [StopTraining],
                        workers=4,
                        verbose=2)


Using real-time data augmentation.
Epoch 1/300
 - 58s - loss: 0.6036 - acc: 0.8484 - val_loss: 0.7596 - val_acc: 0.8291
Epoch 2/300
 - 45s - loss: 0.5894 - acc: 0.8522 - val_loss: 0.8661 - val_acc: 0.8089
Epoch 3/300
 - 46s - loss: 0.5864 - acc: 0.8538 - val_loss: 0.8437 - val_acc: 0.8108
Epoch 4/300
 - 46s - loss: 0.5870 - acc: 0.8552 - val_loss: 0.7108 - val_acc: 0.8367
Epoch 5/300
 - 46s - loss: 0.5841 - acc: 0.8527 - val_loss: 0.7057 - val_acc: 0.8405
Epoch 6/300
 - 46s - loss: 0.5821 - acc: 0.8538 - val_loss: 0.7523 - val_acc: 0.8301
Epoch 7/300
 - 46s - loss: 0.5840 - acc: 0.8525 - val_loss: 0.7844 - val_acc: 0.8241
Epoch 8/300
 - 46s - loss: 0.5797 - acc: 0.8529 - val_loss: 0.6902 - val_acc: 0.8405
Epoch 9/300
 - 46s - loss: 0.5714 - acc: 0.8571 - val_loss: 0.7965 - val_acc: 0.8216
Epoch 10/300
 - 46s - loss: 0.5819 - acc: 0.8554 - val_loss: 0.7765 - val_acc: 0.8250
Epoch 11/300
 - 46s - loss: 0.5792 - acc: 0.8554 - val_loss: 0.6840 - val_acc: 0.8455
Epoch 12/300
 - 46s - loss: 

In [0]:
# Save model and weights
save_dir = '/content/drive/My Drive/Colab Notebook'
model5_name = 'keras_cifar10_trained_model_v6.h5' 
model_path = os.path.join(save_dir, model5_name)
model5.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model5.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at /content/drive/My Drive/Colab Notebook/keras_cifar10_trained_model_v6.h5 
10000/10000 [==============================] - 4s 411us/step
Test loss: 0.6340710081577301
Test accuracy: 0.8468
